In [68]:
#import library

from scipy.spatial import distance as dist
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from sklearn.cluster import DBSCAN
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os
import sys


In [69]:
# below are confguration
USE_GPU = True
MIN_CONF = 0.3
NMS_THRESH = 0.3


Deteksi manusia dan Clustering

In [70]:

def measuring_object_distance(frame, net, ln, personIdx=0):
    (H, W) = frame.shape[:2]
    results = []
    boxes = []
    centroids = []
    confidences = []
    person_coordinates = []

    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
                                 swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    for output in layerOutputs:
        for detection in output:

            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if classID == personIdx and confidence > MIN_CONF:

                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])                
                centroids.append((centerX, centerY))
                confidences.append(float(confidence))

    indexs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)

    if len(indexs) > 0:

        for i in indexs.flatten():

            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            (cx, cy) = centroids[i]
            color = (0, 255, 0)
            cv2.circle(frame, (cx, cy), 5, color, 1)
            results.append([x,y,w,h])
            data = [x,y,w,h]
            person_coordinates.append((data[0]+int(data[2]/2), data[1]+int(data[3]/2)))
        
        clustering = DBSCAN(eps=100,min_samples=2).fit(person_coordinates)
        isSafe = clustering.labels_

        # Count 
        person_count = len(results)
        safe_count = np.sum((isSafe==-1)*1)
        unsafe_count = person_count - safe_count
        print("Person Count: ", person_count)
        print("Safe Count: ", safe_count)

        # buat garis untuk setiap orang yang terdeteksi
        arg_sorted = np.argsort(isSafe)
        for i in range(1,person_count):
          if isSafe[arg_sorted[i]]!=-1 and isSafe[arg_sorted[i]]==isSafe[arg_sorted[i-1]]:
            cv2.line(frame,person_coordinates[arg_sorted[i]],person_coordinates[arg_sorted[i-1]],(0,0,255),2)

        # Put Bounding Boxes on People in the Frame
        for p in range(person_count):
          a,b,c,d = results[p]
          color = (0,255,0) if isSafe[p]==-1 else (0,0,255)
          cv2.rectangle(frame, (a, b), (a + c, b + d), color, 2)        

        xpos = 15

        string = "Total People = "+str(person_count)
        cv2.putText(frame,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
        xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]

        string = " ( "+str(safe_count) + " Safe "
        cv2.putText(frame,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
        xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]

        string = str(unsafe_count)+ " Unsafe ) "
        cv2.putText(frame,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
        xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]
        

Fungsi Deteksi penggunaan masker


In [71]:
def predict_mask(frame, faceNet, maskNet):

    faces = []
    locs = []
    preds = []
    
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
                                 (104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            # add the face and bounding boxes to their respective
            # lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    #hanya lakukan prediksi terdapat wajah
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    return (locs, preds)


Load yolo weights, model dan label


In [72]:
print("[INFO] loading YOLO from disk...")
# load the COCO class labels our YOLO model was trained on
yoloLabels = open("./yolo-coco/coco.names").read().strip().split("\n")
yoloWeigths = "./yolo-coco/yolov3.weights"
yoloConfig = "./yolo-coco/yolov3.cfg"

net = cv2.dnn.readNetFromDarknet(yoloConfig, yoloWeigths)

if USE_GPU:
    print("[INFO] setting preferable backend and target to CUDA...")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)


[INFO] loading YOLO from disk...
[INFO] setting preferable backend and target to CUDA...


Load detector wajah


In [73]:
# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)


[INFO] loading face detector model...


Load model deteksi masker yang sudah di training


In [74]:
print("[INFO] loading face mask detector model...")
MODEL_PATH = "./model"
if not os.path.isdir(MODEL_PATH):
    print("[INFO] Please put the model at model folder")
    sys.exit(0)
maskNet = load_model("./model/mask_detector.h5")


[INFO] loading face mask detector model...


Jalankan deteksi masker dan jarak


In [75]:
print("[INFO] accessing video stream...")
vs = cv2.VideoCapture('./Crowded pedestrian area in Mong Kok.mp4') # un comment this line to use the video
# vs = cv2.VideoCapture(0)

ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

while True:
	(grabbed, frame) = vs.read()

	if not grabbed:
		break

	frame = imutils.resize(frame, width=700)

	measuring_object_distance(frame, net, ln, personIdx=yoloLabels.index("person"))
    
	(locs, preds) = predict_mask(frame, faceNet, maskNet)

	labelsAndColor = [["Pakai Masker", (0, 255, 0)], ["Tidak Pakai Masker", (0, 0, 255)]]
	
	for (box, pred) in zip(locs, preds):
		(startX, startY, endX, endY) = box

		label = labelsAndColor[np.argmax(pred)][0]
		color = labelsAndColor[np.argmax(pred)][1]

		label = "{}: {:.2f}%".format(label, max(pred) * 100)
		print(label)

		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	# show the output frame
	cv2.imshow("Mask And Social Distancing Detector", frame)
	key = cv2.waitKey(1) & 0xFF

	if key == ord("q"):
		break


# do a bit of cleanup
cv2.destroyAllWindows()

[INFO] accessing video stream...
Person Count:  11
Safe Count:  2
Person Count:  11
Safe Count:  2
Person Count:  12
Safe Count:  2
Person Count:  11
Safe Count:  2
Person Count:  10
Safe Count:  1
Person Count:  10
Safe Count:  1
Person Count:  10
Safe Count:  2
Tidak Pakai Masker: 99.98%
Person Count:  12
Safe Count:  2
Tidak Pakai Masker: 100.00%
Person Count:  12
Safe Count:  2
Tidak Pakai Masker: 99.99%
Person Count:  10
Safe Count:  1
Tidak Pakai Masker: 99.91%
Person Count:  13
Safe Count:  1
Tidak Pakai Masker: 100.00%
Person Count:  13
Safe Count:  1
Tidak Pakai Masker: 100.00%
Person Count:  12
Safe Count:  1
Tidak Pakai Masker: 99.99%
Person Count:  12
Safe Count:  1
Tidak Pakai Masker: 100.00%
Person Count:  13
Safe Count:  1
Tidak Pakai Masker: 100.00%
Person Count:  12
Safe Count:  1
Tidak Pakai Masker: 100.00%
Person Count:  10
Safe Count:  1
Tidak Pakai Masker: 99.30%
Person Count:  10
Safe Count:  1
Person Count:  10
Safe Count:  1
Person Count:  10
Safe Count:  1
Tida